In [ ]:
import cupy as cp
import matplotlib.pyplot as plt

# Parámetros
N = 10
Ec = 1.0
El = cp.linspace(0.1, 10, 300)
EDelta = 1.0
phi_ext = 0.0
r_values = cp.linspace(0.1, 10, 300)

phi_ZPF = (2.0 * Ec / El) ** 0.25

def destroy(N):
    return cp.sqrt(cp.arange(1, N))[:, cp.newaxis] * cp.eye(N, k=1)

N_op = 1j * (destroy(N).T - destroy(N)) / phi_ZPF[:, cp.newaxis, cp.newaxis] / 2
phi_op = (destroy(N).T + destroy(N)) * phi_ZPF[:, cp.newaxis, cp.newaxis]

delta = phi_op - phi_ext

ReZ = (phi_op / 2).cos() * (r_values * phi_op / 2).cos()[:, cp.newaxis, cp.newaxis] + r_values[:, cp.newaxis, cp.newaxis] * (phi_op / 2).sin() * (r_values * phi_op / 2).sin()[:, cp.newaxis, cp.newaxis]
ImZ = -(phi_op / 2).cos() * (r_values * phi_op / 2).sin()[:, cp.newaxis, cp.newaxis] + r_values[:, cp.newaxis, cp.newaxis] * (phi_op / 2).sin() * (r_values * phi_op / 2).cos()[:, cp.newaxis, cp.newaxis]

sigmaz = cp.array([[1, 0], [0, -1]])
sigmay = cp.array([[0, -1j], [1j, 0]])

H = 4 * Ec * cp.tensordot(N_op, cp.eye(2), axes=0) ** 2 + 0.5 * El[:, cp.newaxis, cp.newaxis] * cp.tensordot(delta, cp.eye(2), axes=0) ** 2 + EDelta * (cp.tensordot(ReZ, sigmaz, axes=([2, 3], [0, 1])) + cp.tensordot(ImZ, sigmay, axes=([2, 3], [0, 1])))

# Calcula el elemento de matriz <0|n|1>
matrix_element = cp.abs(H[:, 0, N, 1]).reshape(El.size, r_values.size)

# Crear gráfica
El_grid, r_grid = cp.meshgrid(El, r_values, indexing='ij')
plt.contourf(cp.asnumpy(El_grid), cp.asnumpy(r_grid), cp.asnumpy(matrix_element), cmap='viridis')
plt.xlabel('El')
plt.ylabel('r')
plt.colorbar(label='Matrix element')
plt.show()
